# Escenario
La simulación se hará sobre un hospital que contará con las siguientes unidades de atención: Unidad de Cuidados Intensivos (UCI), Unidad de Urgencias, Consultorios. Cada unidad cuenta con recursos limitados: un número específico de camas y un número específico de enfermeras y doctores, y además la Unidad de Urgencias cuenta con un número específico de set de utencilios médicos para cirugías.

## Objetivos
1. Simular la carga de trabajo de cada unidad del hospital y los tiempos de espera de los pacientes. 
2. Sacar conclusiones sobre la eficiencia del manejo de los recursos de cada unidad así como de la suficiencia de los mismos.

In [60]:
import pandas as pd
import numpy as np
import simpy
import random

In [61]:
class Unidad:
    def __init__(self, env, nombre, num_camas, personal, num_kits_medicos):
        self.env = env
        self.nombre = nombre
        self.num_camas = simpy.Resource(env, capacity=num_camas)
        self.personal = simpy.Resource(env, capacity=personal)
        self.num_kits_medicos = simpy.Resource(env, capacity=num_kits_medicos)


In [62]:
class Hospital:
    def __init__(self, env, recepcionistas, unidades):
        self.env = env
        self.unidades = unidades
        self.recepcionistas = simpy.Resource(env, capacity=recepcionistas)

In [63]:
class Paciente:
    def __init__(self, env, id, hospital: Hospital, unidad: Unidad):
        self.env = env
        self.id = id
        self.hospital = hospital
        self.unidad = unidad
        self.cirugia = random.choice([True, False]) if unidad.nombre == "Urgencias" else False
        self.alcanzo_cama = False
        
        self.env.process(self.start())
        self.eventos = pd.DataFrame()
    
    def start(self):
        print(f"{self.env.now}: Paciente {self.id} llega al hospital a la unidad {self.unidad.nombre}.")
        self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id],
                "evento": ["LLEGADA"],
                "time": [self.env.now]
            })])
        
        with self.hospital.recepcionistas.request() as request:
            yield request
            print(f"{self.env.now}: Paciente {self.id} es atendido.")
            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id],
                "evento": ["ATENDIDO"],
                "time": [self.env.now]
            })])

            if self.unidad.nombre == "Urgencias" or self.unidad.nombre == "UCI":
                tiempo_atencion_inferior = 1
                tiempo_atencion_superior = 7
                cant_personal = random.randint(3, 8)
            else:
                tiempo_atencion_inferior = 7
                tiempo_atencion_superior = 25
                cant_personal = 1 #Solo se necesita un doctor para una consulta general

            # El paciente solicita una cama
            with self.unidad.num_camas.request() as cama_request:
                yield cama_request
                with self.unidad.personal.request() as personal_request: # Con comprueba la disponibilidad de personal
                    yield personal_request
                    yield self.env.timeout(random.randint(tiempo_atencion_inferior, tiempo_atencion_superior))
                    print(f"{self.env.now}: Al Paciente {self.id} se le asigno una cama y {cant_personal} miembro(s) del personal.")
                    self.alcanzo_cama = True
                    self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id],
                        "evento": ["ASIGNACION_CAMA_PERSONAL"],
                        "time": [self.env.now]
                    })])

        #Si el paciente es de urgencias y necesita cirugia, espera que se le asigne un kit medico
        if self.cirugia:
            print(f"{self.env.now}: El Paciente {self.id} necesita cirugía.")
            
            # Tiempo máximo que el paciente puede esperar sin morir
            tiempo_max_espera_kit = 60  # en minutos
        
            with self.unidad.num_kits_medicos.request() as kit_request:
                resultado = yield kit_request | self.env.timeout(tiempo_max_espera_kit)

                if kit_request in resultado:
                    # El paciente consiguió el kit a tiempo
                    print(f"{self.env.now}: Al Paciente {self.id} se le asignó un kit médico.")
                    self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id],
                        "evento": ["ASIGNACION_KIT"],
                        "time": [self.env.now]
                    })])
        
                    # Tiempo de cirugía real
                    tiempo_cirugia = random.randint(15, 60 * 8) # Se considera un rango de 15 minutos a 8 horas por cirugía
                    yield self.env.timeout(tiempo_cirugia)
                    print(f"{self.env.now}: Cirugía completada para el Paciente {self.id}.")
        
                    # ... tras cirugía o recuperación …
                    # decisión probabilística: 90% de alta, 10% de muerte
                    prob_supervivencia = 0.9  
                    if random.random() <= prob_supervivencia:
                        evento_final = "ALTA"
                        mensaje = f"{self.env.now}: Paciente {self.id} se le ha dado de alta."
                    else:
                        evento_final = "MUERTE"
                        mensaje = f"{self.env.now}: Paciente {self.id} murió tras el procedimiento."
                    
                    print(mensaje)
                    self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id],
                        "evento": [evento_final],
                        "time": [self.env.now]
                    })])
                else:
                    # El paciente murió esperando un kit médico disponible
                    print(f"{self.env.now}: El Paciente {self.id} murió esperando un kit médico.")
                    self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id],
                        "evento": ["MUERTE"],
                        "time": [self.env.now]
                    })])
                    return
        else:
            # Tiempo de recuperación antes del alta
            tiempo_recuperacion = random.randint(5, 30)
            yield self.env.timeout(tiempo_recuperacion)

            print(f"{self.env.now}: Al Paciente {self.id} se le ha dado de alta.")
            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id],
                "evento": ["ALTA"],
                "time": [self.env.now]
            })])


In [64]:
class Simulation:
    def __init__(self, env, unidades_settings, recepcionistas):
        self.env = env
        self.unidades = [Unidad(env, u["nombre"], u["num_camas"], u["personal"], u["num_kits_medicos"]) for u in unidades_settings]
        self.hospital = Hospital(env, recepcionistas, self.unidades)
        self.pacientes = []

        self.env.process(self.run_simulation())
    
    def run_simulation(self):
        while True:
            yield self.env.timeout(random.randint(1, 5)) # Los pacientes llegan cada 1 a 5 minutos

            # Generar de 1 a 5 pacientes a la vez
            for _ in range(random.randint(1, 5)):
                paciente = Paciente(
                    self.env,
                    len(self.pacientes) + 1,
                    self.hospital,
                    #random.choice(self.unidades)
                    np.random.choice(self.unidades, p=[0.5, 0.3, 0.2])
                )
                self.pacientes.append(paciente)

# Inicio de la simulacion

In [65]:
env = simpy.Environment()
unidades_settings = [
    {"nombre": "Urgencias", "num_camas": 20, "personal": 30, "num_kits_medicos": 8},
    {"nombre": "UCI", "num_camas": 35, "personal": 25, "num_kits_medicos": 1},
    {"nombre": "Consultorios", "num_camas": 10, "personal": 8, "num_kits_medicos": 1}
]
num_recepcionistas = 2

random.seed(111)

simulacion = Simulation(env, unidades_settings, num_recepcionistas)
env.run(until=60*8)

2: Paciente 1 llega al hospital a la unidad UCI.
2: Paciente 2 llega al hospital a la unidad Urgencias.
2: Paciente 3 llega al hospital a la unidad Consultorios.
2: Paciente 1 es atendido.
2: Paciente 2 es atendido.
4: Paciente 4 llega al hospital a la unidad Consultorios.
4: Paciente 5 llega al hospital a la unidad Urgencias.
7: Al Paciente 1 se le asigno una cama y 6 miembro(s) del personal.
7: Paciente 3 es atendido.
8: Paciente 6 llega al hospital a la unidad Urgencias.
8: Paciente 7 llega al hospital a la unidad UCI.
9: Al Paciente 2 se le asigno una cama y 6 miembro(s) del personal.
9: Paciente 4 es atendido.
12: Paciente 8 llega al hospital a la unidad Urgencias.
12: Paciente 9 llega al hospital a la unidad UCI.
12: Paciente 10 llega al hospital a la unidad Consultorios.
12: Paciente 11 llega al hospital a la unidad UCI.
15: Paciente 12 llega al hospital a la unidad Consultorios.
15: Paciente 13 llega al hospital a la unidad UCI.
15: Paciente 14 llega al hospital a la unidad Con

# 4. Recolección de Datos

## Preguntas a responder:
1. ¿Cuál fue el tiempo promedio de espera por unidad?

2. ¿Qué unidad estuvo más congestionada?

3. ¿Cuántos pacientes no fueron atendidos por falta de recursos?

4. ¿Cuál fue el total de pacientes atendidos?

5. ¿Cuál es la cantidad y  % de pacientes que vienen a cada unidad?

6. ¿Qué impacto tuvo cambiar el número de recepcionistas o personal médico?

### Construcción del dataframe de perfil de paciente 

In [66]:
lista_pacientes = []

def registrar_paciente(paciente_id, unidad, cirugia,alcanzo_cama):
    lista_pacientes.append({
        "id": paciente_id,
        "unidad": unidad,
        "cirugia": cirugia,
        "alcanzo_cama": alcanzo_cama
    })

In [67]:
for p in simulacion.pacientes:
    registrar_paciente(p.id, p.unidad.nombre, p.cirugia, p.alcanzo_cama)

dfPacientes = pd.DataFrame(lista_pacientes)
dfPacientes

,id,unidad,cirugia,alcanzo_cama
0,1,UCI,False,True
1,2,Urgencias,False,True
2,3,Consultorios,False,True
3,4,Consultorios,False,True
4,5,Urgencias,True,True
...,...,...,...,...
486,487,UCI,False,False
487,488,Consultorios,False,False
488,489,Consultorios,False,False
489,490,Urgencias,True,False


### Construcción del dataframe de eventos

In [68]:
lista_eventos = []
for p in simulacion.pacientes:
    lista_eventos.append( p.eventos )

dfeventos = pd.concat(lista_eventos)
dfeventos = dfeventos.reset_index()
dfeventos.drop( ["index"], axis=1, inplace=True )

dfeventos

,id,evento,time
0,1,LLEGADA,2
1,1,ATENDIDO,2
2,1,ASIGNACION_CAMA_PERSONAL,7
3,1,ALTA,29
4,2,LLEGADA,2
...,...,...,...
905,487,LLEGADA,478
906,488,LLEGADA,478
907,489,LLEGADA,478
908,490,LLEGADA,478


In [69]:
dfPacientes.loc[ dfPacientes["id"] == 1 , : ]

,id,unidad,cirugia,alcanzo_cama
0,1,UCI,False,True


In [70]:
dfeventos.loc[ dfeventos["id"] == 1 , : ]


,id,evento,time
0,1,LLEGADA,2
1,1,ATENDIDO,2
2,1,ASIGNACION_CAMA_PERSONAL,7
3,1,ALTA,29


## 1. ¿Cuál fue el tiempo promedio de espera por unidad?

In [71]:
dfeventos["lag_time"] = dfeventos.groupby("id")["time"].shift(1)

In [72]:
# Filtramos y calculamos el tiempo de espera
df1 = dfeventos.loc[dfeventos["evento"] == "ATENDIDO", :].copy()
df1["tiempo_espera"] = df1["time"] - df1["lag_time"]

# Merge con la info de pacientes (asegúrate de que 'id' esté en ambos)
df_merged = df1.merge(dfPacientes, on="id", how="inner")

# Agrupamos por unidad y calculamos el promedio
resultado = (
    df_merged
    .groupby("unidad")
    .agg({"tiempo_espera": "mean"})
)

print(resultado)

              tiempo_espera
unidad                     
Consultorios     168.100000
UCI              195.460000
Urgencias        168.016949


## 2. ¿Qué unidad estuvo más congestionada?




In [73]:
# Nombre de la unidad con mayor espera
unidad_max = resultado["tiempo_espera"].idxmax()

# Valor de la espera media más alta
espera_max = resultado["tiempo_espera"].max()
print (f'La unidad más congestionada basandonos en su tiempo medio de atención es {unidad_max} \ncon un tiempo de espera de {espera_max}')

La unidad más congestionada basandonos en su tiempo medio de atención es UCI 
con un tiempo de espera de 195.46


## 3. ¿Cuántos pacientes no fueron atendidos por falta de recursos?

In [74]:
# 1. Pacientes totales que intentaron ser atendidos
todos = set(dfeventos["id"].unique())

# 2. Pacientes que sí consiguieron cama/personal
con_cama = set(dfeventos.loc[dfeventos["evento"] == "ASIGNACION_CAMA_PERSONAL", "id"].unique())

# 3. Pacientes que nunca obtuvieron cama
sin_cama = todos - con_cama

# 4. Pacientes que murieron esperando kit (sin haberse asignado uno)
muertes = set(dfeventos.loc[dfeventos["evento"] == "MUERTE", "id"].unique())
con_kit = set(dfeventos.loc[dfeventos["evento"] == "ASIGNACION_KIT", "id"].unique())
muertes_por_kit = muertes - con_kit

print(f"Pacientes sin cama asignada: {len(sin_cama)}")
print(f"Pacientes que murieron por falta de kits: {len(muertes_por_kit)}")
print(f"Total no atendidos por falta de recursos: {len(sin_cama) + len(muertes_por_kit)}")

Pacientes sin cama asignada: 354
Pacientes que murieron por falta de kits: 7
Total no atendidos por falta de recursos: 361


### 4. ¿Cuál fue el total de pacientes atentidos y cuantos quedaron en recepción?

In [75]:
# Filtrar eventos de tipo 'ATENDIDO'
df_atendidos = dfeventos[dfeventos["evento"] == "ATENDIDO"]

# Contar cuántos pacientes únicos fueron atendidos
total_atendidos = df_atendidos["id"].nunique()

print(f"Total de pacientes atendidos: {total_atendidos}")
print(f"Total de pacientes esperando en recepción o cama: {(len(todos)-total_atendidos)
}")

Total de pacientes atendidos: 139
Total de pacientes esperando en recepción o cama: 352


### 5. ¿Cuál es la cantidad y % de pacientes que vienen a cada unidad?

In [76]:

df_unique = dfPacientes[['id', 'unidad']].drop_duplicates()


df_stats = (
    df_unique['unidad']
    .value_counts()
    .rename_axis('unidad')
    .reset_index(name='total')
)
df_stats['porcentaje'] = df_stats['total'] / df_stats['total'].sum() * 100

print(df_stats)

         unidad  total  porcentaje
0     Urgencias    238   48.472505
1           UCI    155   31.568228
2  Consultorios     98   19.959267


## 5.¿Qué impacto tuvo cambiar el número de recepcionistas o personal médico?

### Aumento de recepcionistas

In [77]:
env2 = simpy.Environment()
unidades_settings = [
    {"nombre": "Urgencias", "num_camas": 20, "personal": 30, "num_kits_medicos": 8},
    {"nombre": "UCI", "num_camas": 35, "personal": 25, "num_kits_medicos": 1},
    {"nombre": "Consultorios", "num_camas": 10, "personal": 8, "num_kits_medicos": 1}
]
num_recepcionistas = 6

random.seed(111)

simulacion2 = Simulation(env2, unidades_settings, num_recepcionistas)
env2.run(until=60*8)



2: Paciente 1 llega al hospital a la unidad UCI.
2: Paciente 2 llega al hospital a la unidad Urgencias.
2: Paciente 3 llega al hospital a la unidad UCI.
2: Paciente 1 es atendido.
2: Paciente 2 es atendido.
2: Paciente 3 es atendido.
4: Paciente 4 llega al hospital a la unidad Consultorios.
4: Paciente 5 llega al hospital a la unidad Consultorios.
4: Al Paciente 2 se le asigno una cama y 6 miembro(s) del personal.
4: Paciente 4 es atendido.
4: Paciente 5 es atendido.
8: Al Paciente 3 se le asigno una cama y 7 miembro(s) del personal.
8: Paciente 6 llega al hospital a la unidad Urgencias.
8: Paciente 7 llega al hospital a la unidad UCI.
8: Paciente 8 llega al hospital a la unidad Urgencias.
8: Paciente 9 llega al hospital a la unidad UCI.
8: Paciente 6 es atendido.
8: Paciente 7 es atendido.
8: Paciente 8 es atendido.
9: Al Paciente 1 se le asigno una cama y 6 miembro(s) del personal.
9: Paciente 9 es atendido.
11: Paciente 10 llega al hospital a la unidad Urgencias.
11: Paciente 11 lle

In [78]:
lista_pacientes2 = []

def registrar_paciente(paciente_id, unidad, cirugia,alcanzo_cama):
    lista_pacientes2.append({
        "id": paciente_id,
        "unidad": unidad,
        "cirugia": cirugia,
        "alcanzo_cama": alcanzo_cama
    })

    

In [79]:
for p in simulacion2.pacientes:
    registrar_paciente(p.id, p.unidad.nombre, p.cirugia, p.alcanzo_cama)

dfPacientes2 = pd.DataFrame(lista_pacientes2)
dfPacientes2

,id,unidad,cirugia,alcanzo_cama
0,1,UCI,False,True
1,2,Urgencias,False,True
2,3,UCI,False,True
3,4,Consultorios,False,True
4,5,Consultorios,False,True
...,...,...,...,...
483,484,Urgencias,False,False
484,485,Urgencias,False,False
485,486,Consultorios,False,False
486,487,Urgencias,False,False


In [80]:
lista_eventos2 = []
for p in simulacion2.pacientes:
    lista_eventos2.append( p.eventos )

dfeventos2 = pd.concat(lista_eventos2)
dfeventos2 = dfeventos2.reset_index()
dfeventos2.drop( ["index"], axis=1, inplace=True )

dfeventos2

,id,evento,time
0,1,LLEGADA,2
1,1,ATENDIDO,2
2,1,ASIGNACION_CAMA_PERSONAL,9
3,1,ALTA,29
4,2,LLEGADA,2
...,...,...,...
1860,484,LLEGADA,476
1861,485,LLEGADA,476
1862,486,LLEGADA,476
1863,487,LLEGADA,476


In [81]:
dfeventos2["lag_time"] = dfeventos2.groupby("id")["time"].shift(1)

In [82]:
# Filtramos y calculamos el tiempo de espera
df2 = dfeventos2.loc[dfeventos2["evento"] == "ATENDIDO", :].copy()
df2["tiempo_espera"] = df2["time"] - df2["lag_time"]

# Merge con la info de pacientes (asegúrate de que 'id' esté en ambos)
df_merged2 = df2.merge(dfPacientes2, on="id", how="inner")

# Agrupamos por unidad y calculamos el promedio
resultado2 = (
    df_merged2
    .groupby("unidad")
    .agg({"tiempo_espera": "mean"})
)

print(resultado2)

              tiempo_espera
unidad                     
Consultorios      12.738095
UCI               14.111888
Urgencias         14.861224


### Resultado
Notamos que el tiempo de espera promedio se reduce considerablemente, indicando que había suficiente personal
médico pero este no era bien aprovechado por el bajo rendimiento laboral de las recepcionistas.

### Aumento de personal

In [83]:
env3 = simpy.Environment()
unidades_settings = [
    {"nombre": "Urgencias", "num_camas": 20, "personal": 60, "num_kits_medicos": 8},
    {"nombre": "UCI", "num_camas": 35, "personal": 50, "num_kits_medicos": 1},
    {"nombre": "Consultorios", "num_camas": 10, "personal": 16, "num_kits_medicos": 1}
]
num_recepcionistas = 2

random.seed(111)

simulacion3 = Simulation(env3, unidades_settings, num_recepcionistas)
env3.run(until=60*8)

2: Paciente 1 llega al hospital a la unidad Consultorios.
2: Paciente 2 llega al hospital a la unidad Consultorios.
2: Paciente 3 llega al hospital a la unidad Urgencias.
2: Paciente 1 es atendido.
2: Paciente 2 es atendido.
4: Paciente 4 llega al hospital a la unidad UCI.
4: Paciente 5 llega al hospital a la unidad Consultorios.
4: Paciente 6 llega al hospital a la unidad Urgencias.
4: Paciente 7 llega al hospital a la unidad UCI.
4: Paciente 8 llega al hospital a la unidad Urgencias.
8: Paciente 9 llega al hospital a la unidad Consultorios.
8: Paciente 10 llega al hospital a la unidad Urgencias.
8: Paciente 11 llega al hospital a la unidad Urgencias.
8: Paciente 12 llega al hospital a la unidad Urgencias.
8: Paciente 13 llega al hospital a la unidad Urgencias.
12: Paciente 14 llega al hospital a la unidad Urgencias.
12: Paciente 15 llega al hospital a la unidad UCI.
12: Paciente 16 llega al hospital a la unidad Consultorios.
14: Paciente 17 llega al hospital a la unidad Urgencias.
14

In [84]:
lista_pacientes3 = []

def registrar_paciente(paciente_id, unidad, cirugia,alcanzo_cama):
    lista_pacientes3.append({
        "id": paciente_id,
        "unidad": unidad,
        "cirugia": cirugia,
        "alcanzo_cama": alcanzo_cama
    })

In [85]:
for p in simulacion3.pacientes:
    registrar_paciente(p.id, p.unidad.nombre, p.cirugia, p.alcanzo_cama)

dfPacientes3 = pd.DataFrame(lista_pacientes3)
dfPacientes3

,id,unidad,cirugia,alcanzo_cama
0,1,Consultorios,False,True
1,2,Consultorios,False,True
2,3,Urgencias,False,True
3,4,UCI,False,True
4,5,Consultorios,False,True
...,...,...,...,...
472,473,Consultorios,False,False
473,474,Urgencias,True,False
474,475,Urgencias,False,False
475,476,Urgencias,True,False


In [86]:
lista_eventos3 = []
for p in simulacion3.pacientes:
    lista_eventos3.append( p.eventos )

dfeventos3 = pd.concat(lista_eventos3)
dfeventos3 = dfeventos3.reset_index()
dfeventos3.drop( ["index"], axis=1, inplace=True )

dfeventos3

,id,evento,time
0,1,LLEGADA,2
1,1,ATENDIDO,2
2,1,ASIGNACION_CAMA_PERSONAL,21
3,1,ALTA,50
4,2,LLEGADA,2
...,...,...,...
898,473,LLEGADA,472
899,474,LLEGADA,472
900,475,LLEGADA,475
901,476,LLEGADA,475


In [87]:
dfeventos3["lag_time"] = dfeventos3.groupby("id")["time"].shift(1)

In [88]:
# Filtramos y calculamos el tiempo de espera
df3 = dfeventos3.loc[dfeventos3["evento"] == "ATENDIDO", :].copy()
df3["tiempo_espera"] = df3["time"] - df3["lag_time"]

# Merge con la info de pacientes (asegúrate de que 'id' esté en ambos)
df_merged3 = df3.merge(dfPacientes3, on="id", how="inner")

# Agrupamos por unidad y calculamos el promedio
resultado3 = (
    df_merged3
    .groupby("unidad")
    .agg({"tiempo_espera": "mean"})
)

print(resultado3)

              tiempo_espera
unidad                     
Consultorios     168.470588
UCI              175.625000
Urgencias        194.426471


## Resultado
Observamos que el tiempo de espera media se mantiene bastante cercano al original, lo que justifica
nuestra respuesta anterior, el mal rendimiento de las recepcionistas evita el correcto uso de recursos
como el del personal médico.